In [37]:
import pandas as pd

In [38]:

df = pd.read_csv('C:/Users/admin/Downloads/dataSet.csv') 
df.head()

,retailer_bill_hash,txn_date,barcode,retailer_id,mrp,sp,qty
0,bc1746a7b50be9d1a2de4f99943359326c3369e99a754e...,2023-03-14,DP,19,90.0,90.0,1.0
1,bc1746a7b50be9d1a2de4f99943359326c3369e99a754e...,2023-03-14,8901719255151,19,10.0,10.0,3.0
2,67addbc1d7dbe8b0c7b63c459ce05259bfff108586c523...,2023-03-17,SS_19_00020,19,4.0,4.0,1.0
3,67addbc1d7dbe8b0c7b63c459ce05259bfff108586c523...,2023-03-17,8904001800275,19,124.0,124.0,1.0
4,67addbc1d7dbe8b0c7b63c459ce05259bfff108586c523...,2023-03-17,8906036670014,19,17.0,17.0,2.0


In [39]:
df.nunique()

retailer_bill_hash    29663
txn_date                 31
barcode               26835
retailer_id              25
mrp                     894
sp                     1598
qty                    1328
dtype: int64

In [40]:
df.columns

Index(['retailer_bill_hash', 'txn_date', 'barcode', 'retailer_id', 'mrp', 'sp',
       'qty'],
      dtype='object')

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   retailer_bill_hash  150000 non-null  object 
 1   txn_date            150000 non-null  object 
 2   barcode             149996 non-null  object 
 3   retailer_id         150000 non-null  int64  
 4   mrp                 150000 non-null  float64
 5   sp                  150000 non-null  float64
 6   qty                 150000 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 8.0+ MB


In [42]:
df.shape

(150000, 7)

In [43]:
df.isnull().sum()

retailer_bill_hash    0
txn_date              0
barcode               4
retailer_id           0
mrp                   0
sp                    0
qty                   0
dtype: int64

In [44]:
# 1)

# Count the number of unique bills
unique_bills = df['retailer_bill_hash'].nunique()

print(f'Total number of unique bills: {unique_bills}')

Total number of unique bills: 29663


In [45]:
# 2)
# Count the number of unique retailers
unique_retailers = df['retailer_id'].nunique()

print(f'Total number of unique retailers: {unique_retailers}')

Total number of unique retailers: 25


In [46]:
# 3)
# Calculate the total sales value
df['total_sales'] = df['sp'] * df['qty']
total_sales_value = df['total_sales'].sum()

# Round to zero decimal places
total_sales_value = round(total_sales_value)

print(f'Total sales value: {total_sales_value}')

Total sales value: 12551209


In [47]:
# 4)
df = df[df['mrp'] > 0].copy()

# Calculate the discount percentage for each transaction
df.loc[:, 'discount_percentage'] = ((df['mrp'] - df['sp']) / df['mrp']) * 100

# Calculate the average discount percentage
average_discount = df['discount_percentage'].mean()

# Round to one decimal place
average_discount = round(average_discount, 1)

print(f'Average discount offered: {average_discount}%')


Average discount offered: -45.1%


In [48]:
# 5)
# Count the number of lines (items) per bill
lines_per_bill = df.groupby('retailer_bill_hash').size()

# Calculate the average lines per bill
average_lines_per_bill = lines_per_bill.mean()

# Round to one decimal place
average_lines_per_bill = round(average_lines_per_bill, 1)

print(f'Average lines per bill: {average_lines_per_bill}')

Average lines per bill: 5.1


In [49]:
# 6)
# Convert the txt_date column to datetime format
df['txn_date'] = pd.to_datetime(df['txn_date'])

# Extract the day of the week (Sunday=0, Monday=1, ..., Saturday=6)
df['day_of_week'] = df['txn_date'].dt.dayofweek

# Calculate the total sales value for each row
df['total_sales'] = df['sp'] * df['qty']

# Calculate the total sales for each day of the week
total_sales_by_day = df.groupby('day_of_week')['total_sales'].sum()

# Identify the day with the maximum sales
max_sales_day = total_sales_by_day.idxmax()

print(f'The day of the week with the maximum sales is: {max_sales_day}')

The day of the week with the maximum sales is: 3


In [50]:
# 7)

def is_compliant(barcode):
    # Convert to string and strip leading zeros
    barcode = str(barcode).lstrip('0')
    
    # Check if the length is 8, 12, or 13
    if len(barcode) not in [8, 12, 13]:
        return False
    
    # Check digit calculation for UPC norms
    # Assuming barcode is numeric
    if not barcode.isdigit():
        return False
    
    def check_digit_upc_e(barcode):
        check_sum = sum((3 if i % 2 == 1 else 1) * int(barcode[i]) for i in range(7))
        return (10 - (check_sum % 10)) % 10 == int(barcode[7])

    def check_digit_upc_a_ean_13(barcode):
        check_sum = sum((3 if i % 2 == len(barcode) % 2 else 1) * int(barcode[i]) for i in range(len(barcode) - 1))
        return (10 - (check_sum % 10)) % 10 == int(barcode[-1])

    if len(barcode) == 8:
        return check_digit_upc_e(barcode)
    elif len(barcode) in [12, 13]:
        return check_digit_upc_a_ean_13(barcode)
    else:
        return False



# the barcode column is treated as string
df['barcode'] = df['barcode'].astype(str)

# Apply the compliance check
df['is_compliant'] = df['barcode'].apply(is_compliant)

# Count the distinct compliant barcodes
distinct_compliant_barcodes = df[df['is_compliant']]['barcode'].nunique()

print(f'Total number of distinct compliant barcodes: {distinct_compliant_barcodes}')


Total number of distinct compliant barcodes: 14505
